<h1>Testing OpenAI API Inference</h1>
This notebook uses the Watchman dataset data. It runs inference using OpenAI service APIs.</br>
This notebook will randomly pick a couple of positive and negative samples to run inference using</br>
"openai-generic" Watchman's model_interfaces.py's model interface. You'll need an OpenAI API account</br>
and API key. The key is expected to be found in the OPENAI_API_KEY variable in your .env file.</br>

---

In [ ]:
import os
import sys
import base64
from pathlib import Path
import pandas as pd
from PIL import Image
from io import BytesIO
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Let's confirm that APIs are working.
# Get the API key from the .env file
parent_dir = os.path.curdir + '/..'
env_path = os.path.join(parent_dir, '.env')
load_dotenv(env_path)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OPENAI_API_KEY has been loaded successfully.")
else:
    print(f"OPENAI_API_KEY not found in the {env_path} file.")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)
models = client.models.list()

print(f"Served models:")
target_model = 'o4-mini' # this one is reasoning, requires a lot of output tokens
#target_model = 'gpt-4o-mini' # faster and no reasoning, but dummer, descriptions are much worse than o4-mini
openai_model = None
for m in models:
    print(f"  {m.id}")
    if m.id == target_model:
        openai_model = m.id
if openai_model:
    print(f"Will use: {openai_model}")
else:
    print(f"No model: {target_model}")

client = None

In [ ]:
# load model interfaces we can use for testing
print(f"Working dir: {os.getcwd()}")
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../orchestrator"))
from shared_settings import *
from model_interfaces import *
print(MODELS)

In [ ]:
# set up vars for finding train data
dataset = "../.data/dataset" # dataset folder location
chans = ["porch"] # list of channels to load the data for
objs = ["person"] # list of objects to load the data for
model_name = "openai-generic" # model interface name to use for inference experimentation
c_desc = {
    "porch": "Porch",
}
o_desc = {
    "person": "a person",
}
MODEL_INTERFACE = MODELS[model_name](model_to_use=openai_model, api_key=openai_api_key)

In [ ]:
# run test inference
from PIL import Image
from IPython.display import display
import base64
import random

def test_inf(s, c, o, res):
    print(f"Subdir: {s} Expecting: {res}")
    image_pname = f"{s}/image.jpg"
    img_data = base64.b64encode(Path(image_pname).read_bytes()).decode()
    res, msg = MODEL_INTERFACE.locate(img_data, o_desc[o], c_desc[c])
    img = Image.open(image_pname)
    w, h = img.size
    display(img.resize((int(w / 4), int(h / 4))))
    print("Inference result: ", "yes" if res else "no")
    print("Location: ", msg if msg is not None else "N/A")
    print("---------------------------")

for c in chans:
    for o in objs:
        dir = f"{dataset}/{c}/{o}"
        subdirs = [f.path for f in os.scandir(dir) if f.is_dir()]
        true_pos = False
        false_pos = False
        random.shuffle(subdirs)
        for s in subdirs:
            if true_pos and false_pos:
                break
            if os.path.exists(f"{s}/skip"):
                continue
            if not false_pos and os.path.exists(f"{s}/no"):
                test_inf(s, c, o, "no")
                false_pos = True
            if not true_pos and not os.path.exists(f"{s}/no"):
                test_inf(s, c, o, "yes")
                true_pos = True
            

---